In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
import numpy as np

## Load Pickles of checkpoint charges per byte

In [3]:
from os import path, pardir, scandir
import pickle
from math import ceil

In [4]:
from pydgilib_extra import LOGGER_CSV

In [5]:
measure_security = False
show_lm_plot=2
measure_workloads = True
repetitions = 100

In [6]:
from experiments.checkpoint_energy import CheckpointEnergy

In [7]:
checkpoint_energy = CheckpointEnergy()

In [8]:
checkpoint_energy.security_projects

['AES', 'AES_Flash', 'TrustZone', 'TrustZone_Flash']

In [9]:
checkpoint_energy.workload_projects

['BitCounting', 'Dijkstra', 'FFT', 'Sorting']

In [10]:
checkpoint_energy.get_device_info(verbose=0)

{'Target voltage': 3.33,
 'Device information': {'Name': 'ATSAML11E16A',
  'JtagId': 'N/A',
  'CPU arch.': 'CORTEX-M23',
  'Series': 'SAML11',
  'DAL': 2},
 'Memory Information': {'base': {'FLASH': [0, 65536],
   'BOCOR': [8437760, 256],
   'SW_CALIB': [8413216, 8],
   'USER_PAGE': [8404992, 256],
   'HPB0': [1073741824, 32768],
   'HPB2': [1107296256, 16384],
   'SCS': [3758153728, 4096]},
  'fuses': {'BOOTROM_BOOTKEY_7': 4294967295,
   'BOOTROM_BOOTKEY_6': 4294967295,
   'BOOTROM_BOOTKEY_5': 4294967295,
   'BOOTROM_BOOTKEY_4': 4294967295,
   'BOOTROM_BOOTKEY_3': 4294967295,
   'BOOTROM_BOOTKEY_2': 4294967295,
   'BOOTROM_BOOTKEY_1': 4294967295,
   'BOOTROM_BOOTKEY_0': 4294967295,
   'BOOTROM_CRCKEY_3': 4294967295,
   'BOOTROM_CRCKEY_2': 4294967295,
   'BOOTROM_CRCKEY_1': 4294967295,
   'BOOTROM_CRCKEY_0': 4294967295,
   'BOOTROM_CEKEY2_3': 4294967295,
   'BOOTROM_CEKEY2_2': 4294967295,
   'BOOTROM_CEKEY2_1': 4294967295,
   'BOOTROM_CEKEY2_0': 4294967295,
   'BOOTROM_CEKEY1_3': 429496

In [11]:
# checkpoint_energy.measure_security_energy('AES_Flash', live_plot=True, max_log_time=200, show_lm_plot=2)

In [12]:
if measure_security:
    checkpoint_energy.measure_all_security_energy(config_dict={"loggers": [LOGGER_CSV]}, show_lm_plot=show_lm_plot)

### Smoothen?

Get lookup function

In [13]:
get_security_energy = checkpoint_energy.get_security_energy_function()

In [14]:
for security_type in checkpoint_energy.security_projects + ["None"]:
    print(security_type, get_security_energy(security_type,100))
    print(security_type, get_security_energy(security_type,100000000))

AES {'AES Encrypt': 4.618429680329748e-06, 'AES Decrypt': 4.760472761953105e-06}
AES {'AES Encrypt': 3.206044320637238, 'AES Decrypt': 3.3284544729845975}
AES_Flash {'AES Encrypt': 5.151138088341776e-06, 'AES Decrypt': 5.331405762945281e-06, 'Flash Write': 6.5229350418467875e-06, 'Flash Read': 1.1769855241474227e-07}
AES_Flash {'AES Encrypt': 3.23922549937155, 'AES Decrypt': 3.343302276044463, 'Flash Write': 2.4002336653118763, 'Flash Read': 0.0767785587343069}
TrustZone {'TrustZone Store': 9.659164384163584e-08, 'TrustZone Load': 1.1363722748402499e-07}
TrustZone {'TrustZone Store': 0.09007525073366786, 'TrustZone Load': 0.08953439418573608}
TrustZone_Flash {'TrustZone Flash Store': 5.752943711745438e-06, 'TrustZone Flash Load': 9.57876867401446e-08}
TrustZone_Flash {'TrustZone Flash Store': 2.573304627986952, 'TrustZone Flash Load': 0.07985659786702963}
None {'None': 0}
None {'None': 0}


In [15]:
if measure_workloads:
    checkpoint_energy.measure_all_workload_energy(live_plot=True, repetitions=repetitions)

Firmware check OK
Chiperase completed successfully

make: Nothing to be done for 'all'.

Firmware check OK
Programming completed successfully.

Starting DGILibExtra with config: 
{'loggers': [1, 2, 0], 'file_name_base': 'experiment_bit_counting', 'plot_pins': [False, False, True, False], 'plot_pins_method': 'line', 'plot_xmax': 0.6, 'window_title': 'Experiment BitCounting', 'log_folder': 'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Workloads\\BitCounting'}

Firmware check OK
Chiperase completed successfully

make: Nothing to be done for 'all'.

Firmware check OK
Programming completed successfully.

Starting DGILibExtra with config: 
{'loggers': [1, 2, 0], 'file_name_base': 'experiment_dijkstra', 'plot_pins': [False, False, True, True], 'plot_pins_method': 'line', 'plot_xmax': 0.6, 'window_title': 'Experiment Dijkstra', 'log_folder': 'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Workloads\\Dijkstra'}

Firmware check OK
Chiperase completed s

In [16]:
checkpoint_energy.measure_workload_energy('Dijkstra', live_plot=True, repetitions=repetitions)

Firmware check OK
Chiperase completed successfully

make: Nothing to be done for 'all'.

Firmware check OK
Programming completed successfully.

Starting DGILibExtra with config: 
{'loggers': [1, 2, 0], 'file_name_base': 'experiment_dijkstra', 'plot_pins': [False, False, True, True], 'plot_pins_method': 'line', 'plot_xmax': 0.6, 'window_title': 'Experiment Dijkstra', 'log_folder': 'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Workloads\\Dijkstra'}



In [17]:
get_workload_energy = checkpoint_energy.get_workload_energy_function()

In [18]:
for workload_type in checkpoint_energy.workload_projects:
#     print(workload_type, len(list(get_workload_energy(workload_type).values())[0]), get_workload_energy(workload_type))
    print(workload_type, len(list(get_workload_energy(workload_type).values())[0]))

BitCounting 100
Dijkstra 100
FFT 100
Sorting 100


In [19]:
# project_path = path.abspath(path.join(path.curdir, "Workloads", "BitCounting", "BitCounting-S"))
# project_path

In [20]:
# checkpoint_energy.get_all_workload_info()

In [21]:
import numpy as np
import matplotlib.pyplot as plt

In [22]:
class WorkloadPlotter(object):
    def __init__(self, checkpoint_energy=CheckpointEnergy(), fig=None, ax=None, show=False):
        self.checkpoint_energy = checkpoint_energy
        self.fig = fig
        self.ax = ax
        if show:
            self.make_fig()
            self.update_fig()
            
        
    def make_fig(self, *args, **kwargs):
        self.fig, self.ax = plt.subplots(*args, **kwargs)
    
    def update_fig(self):
        opacity = 0.8
        self.ax.cla()
        
        self.bars = {}
        for index, workload_project in enumerate(self.checkpoint_energy.workload_projects):
            self.bars[workload_project] = {}
            section_averages = self.checkpoint_energy.get_workload_average(workload_project)
            section_stds = self.checkpoint_energy.get_workload_std(workload_project)
            bottom = 0
            for section in section_averages.keys():
                self.bars[workload_project][section] = self.ax.bar(index, section_averages[section], label=section, alpha=opacity, yerr=section_stds[section], bottom=bottom)
                bottom += section_averages[section]
        
        self.ax.set_xticks(range(len(self.checkpoint_energy.workload_projects)))
        self.ax.set_xticklabels(self.checkpoint_energy.workload_projects)
        self.ax.legend()
        self.ax.set_yscale('log')
        # self.ax.grid()

        self.ax.set_xlabel('Workload')
        self.ax.set_ylabel('Charge [C]')
        self.ax.set_title('Charge per Section')
        self.fig.canvas.set_window_title('Charge per Workload')
        self.fig.show()

In [23]:
workload_plotter = WorkloadPlotter(checkpoint_energy, show=True)

In [24]:
# workload_plotter.make_fig()

In [25]:
# workload_plotter.update_fig()

In [26]:
import matplotlib.animation as animation

In [27]:
class SecurityPlotter(object):
    def __init__(self, checkpoint_energy=CheckpointEnergy(), num_bytes=100, fig=None, ax=None, show=0):
        self.checkpoint_energy = checkpoint_energy
        self.fig = fig
        self.ax = ax
        self.num_bytes = num_bytes
        if show:
            self.make_fig()
            self.update_fig()
            if show >= 2:
                self.animate()
        
    def make_fig(self, *args, **kwargs):
        self.fig, self.ax = plt.subplots(*args, **kwargs)
    
    def update_fig(self, num_bytes=None):
        if num_bytes is None:
            num_bytes = self.num_bytes
        opacity = 0.8
        self.ax.cla()
        
        self.bars = {}
        for index, security_project in enumerate(self.checkpoint_energy.security_projects):
            self.bars[security_project] = {}
            security_energy = self.checkpoint_energy.get_security_energy(security_project, num_bytes)
            bottom = 0
            for section in security_energy.keys():
                self.bars[security_project][section] = self.ax.bar(index, security_energy[section], label=section, alpha=opacity, bottom=bottom)
                bottom += security_energy[section]
        
        self.ax.set_xticks(range(len(self.checkpoint_energy.security_projects)))
        self.ax.set_xticklabels(self.checkpoint_energy.security_projects)
        self.ax.legend()
        # self.ax.set_yscale('log')
        # self.ax.grid()

        self.ax.set_xlabel('Security')
        self.ax.set_ylabel('Charge [C]')
        self.ax.set_title(f'Charge for {num_bytes} bytes')
        self.fig.canvas.set_window_title('Charge per Security')
        self.fig.show()
        
    def anim_func(self, num_bytes, *fargs):
        self.update_fig(num_bytes)
        return flatten(self.bars)
    
    def animate(self, frames=range(100, 10001, 100)):
        self.animation = animation.FuncAnimation(self.fig, self.anim_func, blit=True, interval=0,frames=frames ,repeat=False)
        
def flatten(d, ret=None):
    if ret is None:
        ret = []
    for k, v in sorted(d.items()):
        if isinstance(v, dict):
            flatten(v, ret)
        else:
            ret.extend(v.get_children())
    return ret

In [28]:
security_plotter = SecurityPlotter(checkpoint_energy, show=2)

In [29]:
# security_plotter.make_fig()

In [30]:
# security_plotter.update_fig()

In [31]:
# security_plotter.animate()

In [32]:
# security_plotter.update_fig(1000000)

In [33]:
from math import sqrt

class IterationsPlotter(object):
    def __init__(self, checkpoint_energy=CheckpointEnergy(), num_iterations=3, fig=None, ax=None, show=0):
        self.checkpoint_energy = checkpoint_energy
        self.fig = fig
        self.ax = ax
        self.num_iterations = num_iterations
        self.workload_section_types = {}
        if show:
            self.make_fig()
            self.update_fig()
            if show >= 2:
                self.animate()
        
    def make_fig(self, *args, **kwargs):
        self.fig, self.ax = plt.subplots(*args, **kwargs)
        
        for workload_project in self.checkpoint_energy.workload_projects:
            self.workload_section_types[workload_project] = \
                self.checkpoint_energy.get_config(workload_project)["analysis"]["section_types"]
    
    def update_fig(self, num_iterations=None):
        if num_iterations is None:
            num_iterations = self.num_iterations
        opacity = 0.8

        # ax.grid()
        self.ax.cla()

        self.bars = []
        for index, workload_project in enumerate(self.checkpoint_energy.workload_projects):
            section_averages = self.checkpoint_energy.get_workload_average(workload_project)
            section_stds = self.checkpoint_energy.get_workload_std(workload_project)
            
            bottom = 0
            for section_type, sections in self.workload_section_types[workload_project].items():
                loop_factor = (1 + (section_type == "loop") * (num_iterations-1))
                for section in sections:
                    bar_height = section_averages[section] * loop_factor
                    self.bars.append(self.ax.bar(index, bar_height, label=section, alpha=opacity, yerr=section_stds[section] / sqrt(loop_factor), bottom=bottom))
                    bottom += bar_height
        
        self.ax.set_xticks(range(len(self.checkpoint_energy.workload_projects)))
        self.ax.set_xticklabels(self.checkpoint_energy.workload_projects)
        self.ax.legend()
        self.ax.set_yscale('log')

        self.ax.set_xlabel('Workload')
        self.ax.set_ylabel('Charge [C]')
        self.ax.set_title(f'Charge for {num_iterations} iterations')
        self.fig.canvas.set_window_title('Charge per Workload')
        self.fig.show()
        
    def anim_func(self, num_iterations, *fargs):
        self.update_fig(num_iterations)
        artist_list = []
        for bar in self.bars:
            artist_list += bar
        return artist_list
    
    def animate(self, frames=range(101)):
        self.animation = animation.FuncAnimation(self.fig, self.anim_func, blit=True, interval=0, frames=frames, repeat=False)
#         self.update_fig(max(frames))

In [34]:
iterations_plotter = IterationsPlotter(checkpoint_energy, show=2)

In [35]:
iterations_plotter.update_fig(4000)

D:\Anaconda3\envs\SAML11-32\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in double_scalars


In [36]:
class IterationsPlotter(object):
    def __init__(self, checkpoint_energy=CheckpointEnergy(), num_iterations=3, fig=None, ax=None, show=0):
        self.checkpoint_energy = checkpoint_energy
        self.fig = fig
        self.ax = ax
        self.num_iterations = num_iterations
        self.workload_section_types = {}
        if show:
            self.make_fig()
            self.update_fig()
        
    def make_fig(self, *args, **kwargs):
        self.fig, self.ax = plt.subplots(*args, **kwargs)
        
        for workload_project in self.checkpoint_energy.workload_projects:
            self.workload_section_types[workload_project] = \
                self.checkpoint_energy.get_workload_config(workload_project)["analysis"]["section_types"]
            
        print(self.workload_section_types)
    
    def update_fig(self):
        opacity = 0.8

        # ax.grid()
        self.ax.cla()

        self.bars = []
        for index, workload_project in enumerate(self.checkpoint_energy.workload_projects):
            section_averages = self.checkpoint_energy.get_workload_average(workload_project)
            section_stds = self.checkpoint_energy.get_workload_std(workload_project)
            
            bottom = 0
            for section_type, sections in self.workload_section_types[workload_project].items():
                print(section_type, sections)
                print(type(section_type), section_type, "loop", section_type == "loop")
                for i in range(1 + (section_type == "loop") * (self.num_iterations-1)):
                    print(i)
                    for section in sections:
                        self.bars.append(self.ax.bar(index, section_averages[section], label=section, alpha=opacity, yerr=section_stds[section], bottom=bottom))
                        bottom += section_averages[section]
            
#             section_averages = {}
#             bottoms = [0] * len(workload_energy)
#             for sub_index, section in enumerate(workload_energy.keys()):
#                 section_averages[section] = np.average(workload_energy[section])
#                 self.bars[workload_project][section] = self.ax.bar(index + sub_index * bar_width, section_averages[section], label=section, alpha=opacity, yerr=section_stds[section], bottom=bottoms[sub_index])
#                 bottoms[sub_index] += section_averages[section]

#             # rects += ax.bar(index + i*bar_width, [get_total_charge(application_name, security_type) for application_name in application_names], bar_width, label=security_type, color=colors[i], alpha=opacity)
#         print(self.bars)
        
        self.ax.set_xticks(range(len(self.checkpoint_energy.workload_projects)))
        self.ax.set_xticklabels(self.checkpoint_energy.workload_projects)
        self.ax.legend()
        self.ax.set_yscale('log')

        self.ax.set_xlabel('Workload')
        self.ax.set_ylabel('Charge [C]')
        self.ax.set_title(f'Charge for {self.num_iterations} iterations')
        self.fig.canvas.set_window_title('Charge per Workload')
        self.fig.show()

In [59]:
# checkpoint_energy.get_checkpoint_size("FFT")
checkpoint_energy.get_workload_size("FFT")
# security_type = "AES"
# workload_project = "FFT"
# checkpoint_energy.get_config(security_type)
# checkpoint_energy.get_checkpoint_energy(security_type, workload_project)
# checkpoint_energy.get_checkpoint_size(workload_project)
# checkpoint_energy.get_security_energy(security_type, checkpoint_energy.get_checkpoint_size(workload_project))

{'FFT-S': {'text': 22600,
  'data': 104,
  'bss': 1584,
  'dec': 24288,
  'hex': 24288,
  'filename': 'FFT-S.elf'}}

For example, in the picture above the total RAM usage is  "data" + "bss" = 15 bytes.
In this case, the total checkpoint size will be: RAM usage + 12 GPR  + LR + PC = 29 Bytes.

In [49]:
def get_number_of_checkpoints(workload_charge, cap_size, security_charge):
    if security_charge >= cap_size:
        print(f"Cap size too small for security charge. Got cap_size: {cap_size}, security_charge: {security_charge}")
        return float("inf")
    return workload_charge / (cap_size - security_charge) #if workload_charge > cap_size else workload_charge

In [50]:
class ControllerGUI(object):
    def __init__(self, artists, ax_kwargs = {'facecolor': 'lightgoldenrodyellow'}, left=0.15, width=0.7, space=0.1):
        self.artists = artists
        num = len(artists)
        height = (1-(num+1)*space)/num
        self.fig = plt.figure(figsize=(8,num))

        for i, artist in enumerate(self.artists.values()):
            artist['ax'] = plt.axes([left, space + i*(space+height) , width, height], **ax_kwargs)
            artist['obj'] = artist['args']['type'](artist['ax'], artist['args']['label'], *artist['args']['args'], **artist['args']['kwargs'])

In [55]:
from math import sqrt, floor
import matplotlib as mpl
import matplotlib.animation as animation
import numpy as np

class CapacitorPlotter(object):
    def __init__(self, checkpoint_energy=CheckpointEnergy(), cap_size=1e-5, num_iterations=3, fig=None, ax=None, show=0, axes_switched=True, controller=True):
        self.checkpoint_energy = checkpoint_energy
        self.fig = fig
        self.ax = ax
        self.cap_size = cap_size
        self.num_iterations = num_iterations
        self.security_section_types = {}
        self.workload_section_types = {}
        self.axes_switched = axes_switched
        
        if show:
            self.make_fig()
            self.update_fig()
            if show >= 2:
                self.animate()
        
        if controller:
            artists = {'cap': {'args': {'type': mpl.widgets.Slider, 'label': 'Capacitor size', 'args': [1e-7, 1e-2], 'kwargs': {'valinit': 3e-4, 'valstep': 1e-7, 'valfmt': "%.2e C"}}}, 
                'iterations': {'args': {'type': mpl.widgets.Slider, 'label': 'Iterations', 'args': [0, 100], 'kwargs': {'valinit': 1, 'valstep': 1, 'valfmt': "%d"}}}}

            self.controller = ControllerGUI(artists)
            
            def update(val):
                self.update_fig(self.controller.artists['cap']['obj'].val, int(self.controller.artists['iterations']['obj'].val))
                self.controller.fig.canvas.draw_idle()

            for artist in self.controller.artists.values():
                artist['obj'].on_changed(update)

            
        
    def make_fig(self, *args, **kwargs):
        self.fig, self.ax = plt.subplots(*args, **kwargs)
        self.fig.canvas.mpl_connect('key_press_event', self.resize_fig)
        self.fig.canvas.set_window_title('Charge per Workload')
        
        for security_project in self.checkpoint_energy.security_projects:
            self.security_section_types[security_project] = \
                self.checkpoint_energy.get_config(security_project)["analysis"]["section_types"]
        for workload_project in self.checkpoint_energy.workload_projects:
            self.workload_section_types[workload_project] = \
                self.checkpoint_energy.get_config(workload_project)["analysis"]["section_types"]
            
        self.colormap = plt.get_cmap('prism')
        self.max_num_colors = 15
        self.colors = {}
        self.legend_artists = []
    
    def get_color(self, section_type, hatch):
        if section_type in self.colors:
            color = self.colors[section_type]
        else:
            color = self.colormap(len(self.colors)/self.max_num_colors)
            self.colors[section_type] = color
            self.legend_artists.append(mpl.patches.Patch(facecolor=color, label=section_type, hatch=hatch))
        return color
    
    def resize_fig(self, *args):
        self.update_fig()
    
    def update_fig(self, cap_size=None, num_iterations=None):
        if cap_size is None:
            cap_size = self.cap_size
        else:
            self.cap_size = cap_size
        if num_iterations is None:
            num_iterations = self.num_iterations
        else:
            self.num_iterations = num_iterations
        
        bar_width = 1/(len(self.checkpoint_energy.security_projects) + 1)
        bar_width_factor = 0.9 * bar_width
        opacity = 0.8

        self.ax.cla()
        
        self.bars = []
        for workload_index, workload_project in enumerate(self.checkpoint_energy.workload_projects):
            section_averages = self.checkpoint_energy.get_workload_average(workload_project)
            section_stds = self.checkpoint_energy.get_workload_std(workload_project)
            checkpoint_size = self.checkpoint_energy.get_checkpoint_size(workload_project)
            
            for security_index, security_project in enumerate(self.checkpoint_energy.security_projects):
                bottom = 0
                for workload_section_type, workload_sections in self.workload_section_types[workload_project].items():
                    loop_factor = num_iterations if workload_section_type == "loop" else 1
                    for workload_section in workload_sections:
                        bar_height = section_averages[workload_section] * loop_factor
                        self.bars.append(self.ax.bar(workload_index + security_index * bar_width, bar_height, bar_width_factor, label=workload_section, alpha=opacity, yerr=section_stds[workload_section] / sqrt(loop_factor), bottom=bottom, color=self.get_color(workload_section, 'xx'), hatch='xx'))
                        bottom += bar_height
                
                security_energy = self.checkpoint_energy.get_security_energy(security_project, checkpoint_size)
                number_of_checkpoints = get_number_of_checkpoints(bottom, cap_size, self.checkpoint_energy.get_checkpoint_energy(security_project, workload_project))
#                 print(workload_project, security_project, number_of_checkpoints, floor(number_of_checkpoints))

                for security_section_type, security_sections in self.security_section_types[security_project].items():
                    loop_factor = number_of_checkpoints if security_section_type in ("load", "store") else 1
                    security_bottom = bottom
                    for security_section in security_sections:
                        bar_height = security_energy[security_section] * loop_factor
                        self.bars.append(self.ax.bar(workload_index + security_index * bar_width, bar_height, bar_width_factor, label=security_section, alpha=opacity, bottom=security_bottom, color=self.get_color(security_section, '//'), hatch='//'))
                        security_bottom += bar_height
        
        self.ax.set_xticks(range(len(self.checkpoint_energy.workload_projects)))
        self.ax.set_xticklabels(self.checkpoint_energy.workload_projects)
        self.ax.legend(loc='upper right', handles=self.legend_artists + [mpl.lines.Line2D([0], [0], linestyle='--', color='k', alpha=0.4, label='Checkpoints')])
        
        self.lines = []
        ymin, ymax = self.ax.get_ylim()
        for y in np.arange(ymin, ymax, self.cap_size):
            self.lines.append(self.ax.axhline(y, linestyle='--', color='k', alpha=0.4, zorder=0))
        
        #self.ax.set_yscale('log')
        # self.ax.callbacks.connect("ylim_changed", self.convert_checkpoint_axis)
        self.convert_checkpoint_axis(self.ax)
        
        self.ax.yaxis.set_label_position("right")
        self.ax.yaxis.set_ticks_position("right")
        
        self.ax.set_xlabel('Workload')
        self.ax.set_ylabel('Charge [C]')
        self.ax.set_title(f'Charge for {num_iterations} iterations with capacitor of {cap_size:.2e} C')
        self.switch_axes(self.axes_switched)
        self.fig.show()
        
    def anim_func(self, num_iterations, *fargs):
        self.update_fig(num_iterations=num_iterations)
        artist_list = self.lines
        for bar in self.bars:
            artist_list += bar
        return artist_list
    
    def animate(self, frames=range(1,20)):
        self.animation = animation.FuncAnimation(self.fig, self.anim_func, blit=True, interval=0, frames=frames, repeat=False)
    
    def convert_checkpoint_axis(self, ax):
        """
        Update second axis according with first axis.
        """
        if hasattr(self, "ax_checkpoint"):
            self.ax_checkpoint.remove()
        self.ax_checkpoint = ax.twinx()
        ymin, ymax = ax.get_ylim()
        self.ax_checkpoint.set_ylim(ymin/self.cap_size, ymax/self.cap_size)
        
        self.ax_checkpoint.set_ylabel('Checkpoints')
        #self.ax_checkpoint.set_yscale('log')
        self.ax_checkpoint.tick_params(axis='y', which='minor')
        self.ax_checkpoint.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter("%d"))
        self.ax_checkpoint.yaxis.set_minor_formatter(mpl.ticker.FormatStrFormatter("%d"))
        for tick in self.ax_checkpoint.yaxis.iter_ticks():
            if tick[1] < 1 or round(tick[1]) - tick[1]:
                tick[0].set_visible(False)
    
    def switch_axes(self, axes_switched=None):
        if axes_switched is not None:
            self.axes_switched = axes_switched
        else:
            self.axes_switched = not self.axes_switched
        if self.axes_switched:
            pos = ("right", "left")
        else:
            pos = ("left", "right")
        self.ax.yaxis.set_label_position(pos[0])
        self.ax.yaxis.set_ticks_position(pos[0])
        self.ax_checkpoint.yaxis.set_label_position(pos[1])
        self.ax_checkpoint.yaxis.set_ticks_position(pos[1])

In [56]:
capacitor_plotter = CapacitorPlotter(checkpoint_energy, show=1, cap_size=0.0003, num_iterations=2)

In [42]:
capacitor_plotter.make_fig()

In [44]:
capacitor_plotter.switch_axes()

In [43]:
capacitor_plotter.update_fig(0.0003, 10)

D:\Anaconda3\envs\SAML11-32\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in double_scalars


Cap size too small for security charge. Got cap_size: 0.00021439999999999998, security_charge: 0.00025100758614677454


D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\transforms.py:429: RuntimeWarning: invalid value encountered in double_scalars
  return (x0, y0, x1 - x0, y1 - y0)


Cap size too small for security charge. Got cap_size: 0.0001251, security_charge: 0.00016577251304141313
Cap size too small for security charge. Got cap_size: 0.0001251, security_charge: 0.00016577251304141313
Cap size too small for security charge. Got cap_size: 0.0001251, security_charge: 0.00016577251304141313
Cap size too small for security charge. Got cap_size: 0.0001251, security_charge: 0.00017180180404488212
Cap size too small for security charge. Got cap_size: 0.0001251, security_charge: 0.00025100758614677454
Cap size too small for security charge. Got cap_size: 0.0001251, security_charge: 0.00016577251304141313
Cap size too small for security charge. Got cap_size: 0.0001251, security_charge: 0.00016577251304141313
Cap size too small for security charge. Got cap_size: 0.0001251, security_charge: 0.00016577251304141313
Cap size too small for security charge. Got cap_size: 0.0001251, security_charge: 0.00017180180404488212
Cap size too small for security charge. Got cap_size: 0